# PuLPモジュールを用いた割当問題の最適解の計算
## Pandasを使用しない場合
## PuLPモジュール読み込み

In [1]:
import pulp

## 目的関数の係数を定義
俳優と役を各々リストとして定義する．俳優は俳優1～俳優4なので，俳優の番号を並べたリストActor = [1,2,3,4]で表す．
このリストを作るのに，
    
    Actor = [i+1 for i in range(4)]
を用いている．
range(4)は0から(4-1)の系列を生成する．
役の番号を並べたリストも同様に作る．
配役の評価値Cは，キーを(俳優,役)，値を評価値とする辞書として作成する．

In [2]:
Actor = [i+1 for i in range(4)]
Role = [j+1 for j in range(4)]
C = {(1,1):3, (1,2):6, (1,3):9, (1,4):8, (2,1):6, (2,2):3, (2,3):2, (2,4):4, (3,1):9, (3,2):3, (3,3):2, (3,4):5, (4,1):6, (4,2):2, (4,3):3, (4,4):8}

## 決定変数の定義
(俳優,役)をキーとする辞書として定義する．決定変数は0/1の2値変数なので，cat=pulp.LpBinaryとする．
ActorとRoleの要素の組み合わせに対して変数を定義するため，以下のように for による繰り返しを2重にする．

In [3]:
x ={(i,j):pulp.LpVariable(f'x{i}_{j}', cat=pulp.LpBinary) for i in Actor for j in Role}

## 問題の定義
各決定変数に評価値を乗じて和をとる．決定変数はActorとRoleの要素の組み合わせに対応して定義されているので，総和を求めるにはpulp.lpSum()内で for による繰り返しを2重にする．

各俳優が一役のみ演じる制約を設定する．
for文で繰り返すことにより，俳優の数が増えたときには効率的な記述になる．
同様に，各受取先の受取量を設定する．

In [4]:
p = pulp.LpProblem('割り当て問題', sense=pulp.LpMaximize)
p += pulp.lpSum(C[(i,j)]*x[(i,j)] for i in Actor for j in Role), '目的関数　配役の評価値'

for i in Actor:
    p += pulp.lpSum(x[(i,j)] for j in Role) == 1, f'俳優{i}は一役を演じる制約'

for j in Role:
    p += pulp.lpSum(x[(i,j)] for i in Actor) == 1, f'役{j}は一俳優が演じる制約'
p

割り当て問題:
MAXIMIZE
3*x1_1 + 6*x1_2 + 9*x1_3 + 8*x1_4 + 6*x2_1 + 3*x2_2 + 2*x2_3 + 4*x2_4 + 9*x3_1 + 3*x3_2 + 2*x3_3 + 5*x3_4 + 6*x4_1 + 2*x4_2 + 3*x4_3 + 8*x4_4 + 0
SUBJECT TO
俳優1は一役を演じる制約: x1_1 + x1_2 + x1_3 + x1_4 = 1

俳優2は一役を演じる制約: x2_1 + x2_2 + x2_3 + x2_4 = 1

俳優3は一役を演じる制約: x3_1 + x3_2 + x3_3 + x3_4 = 1

俳優4は一役を演じる制約: x4_1 + x4_2 + x4_3 + x4_4 = 1

役1は一俳優が演じる制約: x1_1 + x2_1 + x3_1 + x4_1 = 1

役2は一俳優が演じる制約: x1_2 + x2_2 + x3_2 + x4_2 = 1

役3は一俳優が演じる制約: x1_3 + x2_3 + x3_3 + x4_3 = 1

役4は一俳優が演じる制約: x1_4 + x2_4 + x3_4 + x4_4 = 1

VARIABLES
0 <= x1_1 <= 1 Integer
0 <= x1_2 <= 1 Integer
0 <= x1_3 <= 1 Integer
0 <= x1_4 <= 1 Integer
0 <= x2_1 <= 1 Integer
0 <= x2_2 <= 1 Integer
0 <= x2_3 <= 1 Integer
0 <= x2_4 <= 1 Integer
0 <= x3_1 <= 1 Integer
0 <= x3_2 <= 1 Integer
0 <= x3_3 <= 1 Integer
0 <= x3_4 <= 1 Integer
0 <= x4_1 <= 1 Integer
0 <= x4_2 <= 1 Integer
0 <= x4_3 <= 1 Integer
0 <= x4_4 <= 1 Integer

## 最適解の計算と結果の読み取り

In [5]:
result = p.solve()

In [6]:
pulp.LpStatus[result]

'Optimal'

In [7]:
pulp.value(p.objective)

29.0

In [8]:
for v in p.variables():
    if pulp.value(v) > 0:
        print(f'{v} = {pulp.value(v):.0f}')

x1_3 = 1
x2_2 = 1
x3_1 = 1
x4_4 = 1


    if pulp.value(v) > 0:
は，変数vの値が0より大きい場合のみ表示させるための条件である．
最適解が x1_3 = 1, x2_2 = 1, x3_1 = 1, x4_4 = 1 となることから，俳優1が役3を，俳優2が役2を，俳優3が役1を，俳優4が役4を演じるのが評価がの和が最も高く，そのときの評価値の和は29である．